# 酵母画像特徴量データから、細胞状態の予測

ここでは、サポートベクターマシン（SVM）とRandom Forestを用いて、酵母の特徴量のデータから芽の大きさを予測します。細胞周期の予測を意図していますが、一旦芽の大きさのカテゴリを当て、細胞周期は、演習問題とします。

SCMDで芽の大きさは４つのカテゴリに分かれています。一つは、細胞が正しく同定出来なかった場合（complex）なので、残りの4クラスのいずれであるかを予測します。以下は、SCMDでの定義です。分ける基準になっているC118は利用を避けます。
![BudSize](img/scmd_budsize.png)

## データの読み込みと正規化

準備として、「0. 酵母画像解析（データ準備〜可視化〜PCA）」で利用していた酵母の細胞周期のデータを読み込み、正規化します。

In [1]:
import numpy as np
import pandas as pd
from sklearn import preprocessing

# データの読み込み
data = pd.read_csv("data/yeast_his3.csv")
columns = ["C101", "C103", "C104","C115","A101","A120","A121","A122","A123"]
cell_features_pre = data[["Cgroup"] + columns]
cell_features = cell_features_pre[np.sum(cell_features_pre.isnull(), axis=1) == 0]
# 正規化
X = cell_features[columns] 
X_norm = preprocessing.StandardScaler().fit_transform(X)

今回は芽の大きさ("Cgroup")の予測を実施しますが、現在"Cgroup" の値は、"large", "medium", "small", "no"の4種類になっています。一方で、機械学習ライブラリが扱える値は文字列ではなく、数値(0,1,2,3)で表されたクラスなので、文字列→数値の変換を実施します。

In [2]:
y = np.zeros(X.shape[0])
y[cell_features["Cgroup"] == "medium"] = 1
y[cell_features["Cgroup"] == "small"] = 2
y[cell_features["Cgroup"] == "no"] = 3

正しく変換されたことを確認するため、変換の前後で各クラスの個数をカウントします。比較的、いずれのクラスも値が存在し、クラス間のバランスが良い印象を得ます。

In [3]:
import collections
collections.Counter(np.array(cell_features["Cgroup"]))

Counter({'medium': 88, 'no': 114, 'large': 97, 'small': 73})

In [4]:
collections.Counter(y)

Counter({1.0: 88, 3.0: 114, 0.0: 97, 2.0: 73})

## データの分割

機械学習では、データを２分割し、片方でモデルの学習を実施、もう一方（モデルに利用しなかったデータ）で予測精度の検証を行います。これにより、学習結果が汎用的か（過学習しているかどうか）の判定を行います。
ここでは、全体の2/3の細胞を利用して学習を実施し、残りの1/3の細胞で予測精度の検証を行います。前者のデータを訓練データ（ training data）、後者のデータをテストデータ（test data）と呼びます。

データ分割は、scikit learn に実装されているため、以下のコマンド一つで実施できます。

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_norm, y, test_size=0.33)

分割されたデータ数を確認します。

In [6]:
print("X: %s\tX_train: %s\tX_test: %s" % (X_norm.shape, X_train.shape, X_test.shape))
print("y: %d\ty_train:%d\ty_test:%d" % (len(y), len(y_train), len(y_test)))

X: (372, 9)	X_train: (249, 9)	X_test: (123, 9)
y: 372	y_train:249	y_test:123


In [7]:
collections.Counter(y_train)

Counter({0.0: 68, 1.0: 60, 3.0: 76, 2.0: 45})

In [8]:
collections.Counter(y_test)

Counter({3.0: 38, 2.0: 28, 1.0: 28, 0.0: 29})

## SVMの実施

Pythonの機械学習ライブラリscikit learnを利用し、芽の大きさのクラス分類するSVMのモデルを学習します。[Sckit-learnのSVMのモジュール](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.svm)にSVMのアルゴリズムが複数実装されています。ここでは、その中でもクラス分類問題向けに作られている関数 SVCを利用します。

本来であれば、カーネル関数を何にするか、SVMの式中に現れるペナルティCをいくつに設定するかなどのハイパーパラメータを最適化する必要がありますが、ここでは全体の手順のみに注力するため、ハイパーパラメータの最適化は実施しません。

In [9]:
from sklearn.svm import SVC
clf = SVC(gamma='auto') #　実行するクラス分類機の設定。ここではSVM
clf.fit(X_train, y_train) # クラス分類の実行、モデルの作成

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

作成したモデルを基に、テストデータ上での正答率を計測します。ここでは、確認のため、同時に訓練データでの正答率も確認します。
訓練ーテストデータの分割での、学習時にも乱数を利用しているため、皆さんのPCで実行した場合に、正答率が異なる可能性があります。

In [10]:
# 訓練データでの精度（参考）
y_train_pred = clf.predict(X_train)
print("Training accuracy: ", sum(y_train_pred == y_train) / len(y_train))
# テストデータでの精度（本番）
y_test_pred = clf.predict(X_test)
print("Test accuracy: ", sum(y_test_pred == y_test) / len(y_test))

Training accuracy:  0.8594377510040161
Test accuracy:  0.7723577235772358


このaccuracyは、正しいクラスに分類された結果しか評価できません。実際には "large"を"no"に間違えるのか、"large"を"medium"に間違えるのかで、意味合いが異なる可能性があります。どのように間違えているのかを、分割表を書くことで確認します。

pandasに変換しているのは、表をキレイに表示するためだけで、本質的な意味はありません。

In [11]:
from sklearn.metrics import confusion_matrix
targets = ['large', 'medium', 'small', 'no']
pd.DataFrame(confusion_matrix(y_train, y_train_pred), columns=targets) # Training

,large,medium,small,no
0,60,8,0,0
1,5,51,4,0
2,0,5,29,11
3,0,0,2,74


In [12]:
pd.DataFrame(confusion_matrix(y_test, y_test_pred), columns=targets) # Test

,large,medium,small,no
0,25,4,0,0
1,2,20,1,5
2,0,1,18,9
3,0,0,6,32


この分割表から、間違えていても多くは近い値であって、"large" を "no" に間違えるようなことは、起こっていないと分かります。

## Random Forest の実施

SVM同様に、Random Forest を実施します。変更点は、SVCがRandomForestClassifier になるだけです。

In [13]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=10, max_depth=5)
clf.fit(X_norm,y)
# 訓練データでの精度（参考）
y_train_pred = clf.predict(X_train)
print("Training accuracy: ", sum(y_train_pred == y_train) / len(y_train))
# テストデータでの精度（本番）
y_test_pred = clf.predict(X_test)
print("Test accuracy: ", sum(y_test_pred == y_test) / len(y_test))

Training accuracy:  0.8875502008032129
Test accuracy:  0.8943089430894309


SVMに比べて、高い精度を出しているように見えます。分割表を書いてみましょう。

In [14]:
pd.DataFrame(confusion_matrix(y_train, y_train_pred), columns=targets) # Training

,large,medium,small,no
0,62,4,2,0
1,6,51,2,1
2,0,4,33,8
3,0,0,1,75


In [15]:
pd.DataFrame(confusion_matrix(y_test, y_test_pred), columns=targets) # Test

,large,medium,small,no
0,26,3,0,0
1,2,21,3,2
2,0,2,26,0
3,0,0,1,37


SVM同様に大きな誤りは少なく、高い精度で予測できていることがわかります。

以上で、SVMとRandomForestを利用して、酵母の画像から得られた特徴量を利用して、細胞周期（芽の大きさ）が、ある程度分類できることがわかりました。次は、深層学習を利用して細胞周期を予測します。